# Flatten tiles

# table of content
1) [Upsample](#upsample)
2) [Downsample](#random-downsampling)

### Dependencies and general utils

In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import open3d as o3d
import laspy
import pdal
import json
from plyfile import PlyData, PlyElement
from tqdm import tqdm
import pdal

In [7]:
src_file = r"D:\GitHubProjects\PointCloudUtils\data\test_densification\SwissSurface_2019_2538_1154_tile_72.pcd"
out_src = src_file.split('.pcd')[0] + "_densify.pcd"
pcd = o3d.io.read_point_cloud(src_file)

In [10]:
# Densification
upsample_factor = 4
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=1.0, max_nn=30))
    
# Reconstruct a surface using Poisson reconstruction
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=10)

# Sample points uniformly on the reconstructed surface
densified_pcd = mesh.sample_points_uniformly(number_of_points=len(pcd.points) * upsample_factor)

# Save the new, denser point cloud
o3d.io.write_point_cloud(out_src, densified_pcd)

True

In [9]:
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=10)
densified = mesh.sample_points_uniformly(number_of_points=len(pcd.points)*4)
o3d.io.write_point_cloud(out_src, mls)

RuntimeError: [Open3D Error] (class std::tuple<class std::shared_ptr<class open3d::geometry::TriangleMesh>,class std::vector<double,class std::allocator<double> > > __cdecl open3d::geometry::TriangleMesh::CreateFromPointCloudPoisson(const class open3d::geometry::PointCloud &,unsigned __int64,float,float,bool,int)) D:\bld\open3d_1731617939957\work\cpp\open3d\geometry\SurfaceReconstructionPoisson.cpp:732: Point cloud has no normals


### Random downsampling

In [8]:
# Utils

def random_downsample_laz(input_path, output_path, fraction=0.1, seed=42):
    """
    Randomly downsamples a LAZ file using PDAL's filters.sample.
    
    Parameters:
        input_path (str): Path to the input LAZ file.
        output_path (str): Path to the output LAZ file.
        fraction (float): Fraction of points to keep (e.g. 0.1 keeps 10% of points).
        seed (int): Random seed for reproducibility.
    """
    # Load metadata to get total point count (for slicing)
    info = pdal.Pipeline(json.dumps([input_path, {"type": "filters.info"}]))
    info.execute()
    total_points = info.metadata["metadata"]["readers.las"]["count"]
    keep_points = int(total_points * fraction)
    
    pipeline_json = {
        "pipeline": [
            input_path,
            {"type": "filters.randomize", "seed": seed},
            {"type": "filters.head", "count": keep_points},
            {"type": "writers.las", "filename": output_path}
        ]
    }

    # Run the pipeline
    pipeline = pdal.Pipeline(json.dumps(pipeline_json))
    count = pipeline.execute()

    print(f"✅ Downsampled {os.path.basename(input_path)} → {os.path.basename(output_path)}")
    print(f"   Kept ≈ {fraction*100:.1f}% of points ({count} written)")

In [11]:
downsample_factor = 0.1
src_file = r"D:\GitHubProjects\Terranum_repo\TreeSegmentation\ForestFormer3D\data\ForAINetV2\test_data\other_small_test.laz"
src_output = src_file.split('.laz')[0] + f"_downsampled_factor_{downsample_factor}.laz"
random_downsample_laz(src_file, src_output, downsample_factor)

✅ Downsampled other_small_test.laz → other_small_test_downsampled_factor_0.1.laz
   Kept ≈ 10.0% of points (47350 written)
